In [30]:
import pandas as pd
import os
import env
import matplotlib.pyplot as plt
import pandas as pd
from pydataset import data
from acquire import get_telco_data
import warnings
warnings.filterwarnings('ignore')
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [31]:
df = get_telco_data()

In [32]:
df = df.loc[:,~df.columns.duplicated()]

In [33]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,Yes,3,Yes,2,90.45,5957.9,No,Two year,DSL,Mailed check
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,No,3,No,4,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,No,3,No,3,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,No,3,Yes,4,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,Yes,3,Yes,3,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [34]:
def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [35]:
train, validate, test = train_validate_test_split(df, target='churn')
train.head(2)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
5642,1982-FEBTD,Female,0,Yes,Yes,23,Yes,Yes,3,No internet service,...,No internet service,3,Yes,4,25.6,514.75,No,Two year,None,Credit card (automatic)
3338,8982-NHAVY,Male,0,No,No,27,Yes,Yes,2,No,...,Yes,2,Yes,3,100.5,2673.45,No,One year,Fiber optic,Bank transfer (automatic)


In [36]:
train.shape, validate.shape, test.shape

((3943, 24), (1691, 24), (1409, 24))

In [37]:
X_train, y_train = train.drop(columns='churn'), train.churn
X_validate, y_validate = validate.drop(columns='churn'), validate.churn
X_test, y_test = test.drop(columns='churn'), test.churn

In [38]:
X_train.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,contract_type,internet_service_type,payment_type
5642,1982-FEBTD,Female,0,Yes,Yes,23,Yes,Yes,3,No internet service,...,No internet service,No internet service,3,Yes,4,25.60,514.75,Two year,None,Credit card (automatic)
3338,8982-NHAVY,Male,0,No,No,27,Yes,Yes,2,No,...,Yes,Yes,2,Yes,3,100.50,2673.45,One year,Fiber optic,Bank transfer (automatic)
4273,4115-NZRKS,Female,1,No,No,7,Yes,No,2,No,...,Yes,Yes,1,Yes,1,89.15,574.35,Month-to-month,Fiber optic,Electronic check
2382,9633-DENPU,Female,0,Yes,No,47,Yes,No,1,No,...,Yes,No,1,Yes,3,58.60,2723.4,Month-to-month,DSL,Bank transfer (automatic)
2655,5693-PIPCS,Male,0,No,No,41,Yes,No,2,No,...,Yes,Yes,3,Yes,4,99.65,4220.35,Two year,Fiber optic,Credit card (automatic)


In [39]:
X_train = X_train.drop(columns = ['customer_id', 'gender'])

In [40]:
dummy_df = pd.get_dummies(df[['gender']], drop_first=True)
df = pd.concat([df, dummy_df], axis=1)

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import graphviz
from graphviz import Graph
from sklearn import tree
import numpy
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [42]:
clf1 = DecisionTreeClassifier(max_depth=3, random_state=123)

In [43]:
clf1 = clf1.fit(X_train, y_train)

ValueError: could not convert string to float: 'Yes'